In [1]:
import numpy as np

import torch
import torchvision
import torch.optim as optim
import torch.nn as nn

import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd

import tqdm
from tqdm import tqdm

import random
import pickle
import time

import pickle

In [2]:
#loading data



# allAnomaly = np.load("allAnomaly3.npy", allow_pickle=True)

# test = allNormal[:]
# test = test.tolist()

# for i in allAnomaly:
#     test.append(i)

# print(len(test))

In [3]:
def makeWindows(data, numWindows):
    windows = []
    intervalLength = int(430 / numWindows)
    for mspec in range(len(data)):
        start = 0
        end = intervalLength
        for interval in range(numWindows):
            melspectrogram = data[mspec]
            window = melspectrogram[:,start:end]
            windows.append(window)
            start += intervalLength
            end += intervalLength
    return windows
        

In [4]:
def makeWindows2(data, numWindows):
    windows = []
    intervalLength = int(430 / numWindows)
    for mspec in tqdm(range(len(data))):
        start = 0
        end = intervalLength
        for interval in range(numWindows):
            melspectrogram = data[mspec]
            window = melspectrogram[:,start:end]
            windows.append([window, mspec, interval])
            start += intervalLength
            end += intervalLength
    return windows
    

In [5]:
class CAE(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Sequential( #starting size: (11,10)
            nn.Conv2d(1,32,3), #input: 11, 10 -> 9, 8
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,3), #input: 4, 4 -> 2,2
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2) #input: 2, 2 -> 1, 1
            #final: 1, 1
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, stride=2), #input: 1,1 -> 3,3
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 6, stride=2, output_padding = (1,0)) #input: 3, 3 -> 11,10
            
            #final: 11,10
        )
        
    def forward(self,x):
        x = self.encoder(x)
        x=self.decoder(x)
        return x

In [6]:
# windows[0].shape

In [7]:
# test = CAE()

# o = test.encoder(torch.from_numpy(windows[0].reshape(-1,1,11,86)).type(torch.float32))
# o.shape

In [8]:
# len(BATCHES[0][0][0])

In [9]:
# batched_training_data.shape

In [10]:
# y = allNormal[:5, 0]
# y2 = makeWindows(y, 5)
# y2= np.array(y2)
# np.random.seed(2345)
# np.random.shuffle(y2)
# print(y2.dtype)
# y2 = torch.from_numpy(y2).type(torch.float32)
# print(type(y2))


In [11]:
#training
data = np.load("dev_fan.npy")
allNormal = np.load("dev_test_fanNormal.npy", allow_pickle=True)

#taking a sample size of 10,000
#8:2 split (training_data:testing_data ratio)
np.random.seed(1234)
np.random.shuffle(data)
training_data = data[:] #length of 8000
# testing_data = data[8000:10000] #length of 2000

windows = makeWindows(training_data, 43) #this makes the window size 11 x 10
# testWindows = makeWindows(testing_data, 5)

#creating the batches - fan
#total windows will be 247,938
BATCH_SIZE = 31
BATCHES = []

copy = windows

seed = 0

iterations = int((len(copy))/BATCH_SIZE)
for i in range(iterations):
    random.seed(seed)
    start = random.randint(0, len(copy) - BATCH_SIZE)
    seed += 1
    end = start + BATCH_SIZE
    batch = copy[start:end]
    BATCHES.append(batch)
    copy = copy[:start] + copy[end:] #taking out the batch from the training_data
    #to prevent duplicate numbers from appearing across batches
   
print(len(copy))
print(f'Shape: {len(BATCHES)}, {len(BATCHES[0])}')


#converting the python list to tensor

batched_training_data = np.array(BATCHES)
np.random.seed(2345)
np.random.shuffle(batched_training_data)
batched_training_data = torch.from_numpy(batched_training_data).type(torch.float32)

model = CAE()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

size = 31

allNormalCut = allNormal[:31, 0]
allNormalCut = makeWindows(allNormalCut, 43)
allNormalCut = allNormalCut[:size]
allNormalCut = np.array(allNormalCut)
 
NUM_OF_EPOCH = 50

train_losses = []
val_losses = []

#outputs = []

def test(testX):
    testX = torch.from_numpy(testX).type(torch.float32)
    outputs = model(testX.view(-1,1,11,10))
    val_loss = criterion(outputs.view(-1,1,11,10), testX.view(-1,1,11,10))
    return val_loss

with open("fan_model1.log", "a") as f:
    for epoch in range(NUM_OF_EPOCH):
        for i in range(len(batched_training_data)):
            output = model(batched_training_data[i].view(-1,1,11,10))
            loss = criterion(output.view(-1,1,11,10), batched_training_data[i].view(-1,1,11,10))
#             train_losses.append(loss.item())
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            if i % 50 == 0:
                vloss = test(allNormalCut)
                val_losses.append(vloss)
                f.write(f"fan_1,{round(time.time(),3)},{round(float(loss.item()), 4)},{round(float(vloss),4)}\n")
    #outputs.append((epoch, output, batched_training_data[i]))
    print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
#     np.save("training_Losses17.npy", train_losses)



filename = 'fan_1.pt'
pickle.dump(model, open(filename, 'wb'))

8
Shape: 9045, 31
Epoch 49, Loss: 3.7697


In [12]:
#training
data = np.load("dev_pump.npy")
allNormal = np.load("dev_test_pumpNormal.npy", allow_pickle=True)

#taking a sample size of 10,000
#8:2 split (training_data:testing_data ratio)
np.random.seed(1234)
np.random.shuffle(data)
training_data = data[:] #length of 8000
# testing_data = data[8000:10000] #length of 2000

windows = makeWindows(training_data, 43) #this makes the window size 11 x 10
# testWindows = makeWindows(testing_data, 5)

#creating the batches - fan
#total windows will be 247,938
BATCH_SIZE = 31
BATCHES = []

copy = windows

seed = 0

iterations = int((len(copy))/BATCH_SIZE)
for i in range(iterations):
    random.seed(seed)
    start = random.randint(0, len(copy) - BATCH_SIZE)
    seed += 1
    end = start + BATCH_SIZE
    batch = copy[start:end]
    BATCHES.append(batch)
    copy = copy[:start] + copy[end:] #taking out the batch from the training_data
    #to prevent duplicate numbers from appearing across batches
   
print(len(copy))
print(f'Shape: {len(BATCHES)}, {len(BATCHES[0])}')


#converting the python list to tensor

batched_training_data = np.array(BATCHES)
np.random.seed(2345)
np.random.shuffle(batched_training_data)
batched_training_data = torch.from_numpy(batched_training_data).type(torch.float32)

model = CAE()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

size = 31

allNormalCut = allNormal[:31, 0]
allNormalCut = makeWindows(allNormalCut, 43)
allNormalCut = allNormalCut[:size]
allNormalCut = np.array(allNormalCut)

NUM_OF_EPOCH = 50

train_losses = []
val_losses = []

#outputs = []

def test(testX):
    testX = torch.from_numpy(testX).type(torch.float32)
    outputs = model(testX.view(-1,1,11,10))
    val_loss = criterion(outputs.view(-1,1,11,10), testX.view(-1,1,11,10))
    return val_loss

with open("pump_model1.log", "a") as f:
    for epoch in range(NUM_OF_EPOCH):
        for i in range(len(batched_training_data)):
            output = model(batched_training_data[i].view(-1,1,11,10))
            loss = criterion(output.view(-1,1,11,10), batched_training_data[i].view(-1,1,11,10))
#             train_losses.append(loss.item())
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            if i % 50 == 0:
                vloss = test(allNormalCut)
                val_losses.append(vloss)
                f.write(f"pump_1,{round(time.time(),3)},{round(float(loss.item()), 4)},{round(float(vloss),4)}\n")
    #outputs.append((epoch, output, batched_training_data[i]))
    print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
#     np.save("training_Losses17.npy", train_losses)



filename = 'pump_1.pt'
pickle.dump(model, open(filename, 'wb'))

0
Shape: 7998, 31
Epoch 49, Loss: 4.4284


In [13]:
#training
data = np.load("dev_slider.npy")
allNormal = np.load("dev_test_sliderNormal.npy", allow_pickle=True)

#taking a sample size of 10,000
#8:2 split (training_data:testing_data ratio)
np.random.seed(1234)
np.random.shuffle(data)
training_data = data[:] #length of 8000
# testing_data = data[8000:10000] #length of 2000

windows = makeWindows(training_data, 43) #this makes the window size 11 x 10
# testWindows = makeWindows(testing_data, 5)

#creating the batches - fan
#total windows will be 247,938
BATCH_SIZE = 31
BATCHES = []

copy = windows

seed = 0

iterations = int((len(copy))/BATCH_SIZE)
for i in range(iterations):
    random.seed(seed)
    start = random.randint(0, len(copy) - BATCH_SIZE)
    seed += 1
    end = start + BATCH_SIZE
    batch = copy[start:end]
    BATCHES.append(batch)
    copy = copy[:start] + copy[end:] #taking out the batch from the training_data
    #to prevent duplicate numbers from appearing across batches
   
print(len(copy))
print(f'Shape: {len(BATCHES)}, {len(BATCHES[0])}')


#converting the python list to tensor

batched_training_data = np.array(BATCHES)
np.random.seed(2345)
np.random.shuffle(batched_training_data)
batched_training_data = torch.from_numpy(batched_training_data).type(torch.float32)

model = CAE()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

size = 31

allNormalCut = allNormal[:31, 0]
allNormalCut = makeWindows(allNormalCut, 43)
allNormalCut = allNormalCut[:size]
allNormalCut = np.array(allNormalCut)

NUM_OF_EPOCH = 50

train_losses = []
val_losses = []

#outputs = []

def test(testX):
    testX = torch.from_numpy(testX).type(torch.float32)
    outputs = model(testX.view(-1,1,11,10))
    val_loss = criterion(outputs.view(-1,1,11,10), testX.view(-1,1,11,10))
    return val_loss

with open("slider_model1.log", "a") as f:
    for epoch in range(NUM_OF_EPOCH):
        for i in range(len(batched_training_data)):
            output = model(batched_training_data[i].view(-1,1,11,10))
            loss = criterion(output.view(-1,1,11,10), batched_training_data[i].view(-1,1,11,10))
#             train_losses.append(loss.item())
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            if i % 50 == 0:
                vloss = test(allNormalCut)
                val_losses.append(vloss)
                f.write(f"slider_1,{round(time.time(),3)},{round(float(loss.item()), 4)},{round(float(vloss),4)}\n")
    #outputs.append((epoch, output, batched_training_data[i]))
    print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
#     np.save("training_Losses17.npy", train_losses)



filename = 'slider_1.pt'
pickle.dump(model, open(filename, 'wb'))

26
Shape: 7176, 31
Epoch 49, Loss: 4.8440


In [14]:
#training
data = np.load("dev_ToyCar.npy")
allNormal = np.load("dev_test_ToyCarNormal.npy", allow_pickle=True)

#taking a sample size of 10,000
#8:2 split (training_data:testing_data ratio)
np.random.seed(1234)
np.random.shuffle(data)
training_data = data[:] #length of 8000
# testing_data = data[8000:10000] #length of 2000

windows = makeWindows(training_data, 43) #this makes the window size 11 x 10
# testWindows = makeWindows(testing_data, 5)

#creating the batches - fan
#total windows will be 247,938
BATCH_SIZE = 31
BATCHES = []

copy = windows

seed = 0

iterations = int((len(copy))/BATCH_SIZE)
for i in range(iterations):
    random.seed(seed)
    start = random.randint(0, len(copy) - BATCH_SIZE)
    seed += 1
    end = start + BATCH_SIZE
    batch = copy[start:end]
    BATCHES.append(batch)
    copy = copy[:start] + copy[end:] #taking out the batch from the training_data
    #to prevent duplicate numbers from appearing across batches
   
print(len(copy))
print(f'Shape: {len(BATCHES)}, {len(BATCHES[0])}')


#converting the python list to tensor

batched_training_data = np.array(BATCHES)
np.random.seed(2345)
np.random.shuffle(batched_training_data)
batched_training_data = torch.from_numpy(batched_training_data).type(torch.float32)

model = CAE()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

size = 31

allNormalCut = allNormal[:31, 0]
allNormalCut = makeWindows(allNormalCut, 43)
allNormalCut = allNormalCut[:size]
allNormalCut = np.array(allNormalCut)

NUM_OF_EPOCH = 50

train_losses = []
val_losses = []

#outputs = []

def test(testX):
    testX = torch.from_numpy(testX).type(torch.float32)
    outputs = model(testX.view(-1,1,11,10))
    val_loss = criterion(outputs.view(-1,1,11,10), testX.view(-1,1,11,10))
    return val_loss

with open("ToyCar_model1.log", "a") as f:
    for epoch in range(NUM_OF_EPOCH):
        for i in range(len(batched_training_data)):
            output = model(batched_training_data[i].view(-1,1,11,10))
            loss = criterion(output.view(-1,1,11,10), batched_training_data[i].view(-1,1,11,10))
#             train_losses.append(loss.item())
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            if i % 50 == 0:
                vloss = test(allNormalCut)
                val_losses.append(vloss)
                f.write(f"ToyCar_1,{round(time.time(),3)},{round(float(loss.item()), 4)},{round(float(vloss),4)}\n")
    #outputs.append((epoch, output, batched_training_data[i]))
    print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
#     np.save("training_Losses17.npy", train_losses)



filename = 'ToyCar_1.pt'
pickle.dump(model, open(filename, 'wb'))

21
Shape: 9709, 31
Epoch 49, Loss: 1.9353


In [16]:
#training
data = np.load("dev_Toy_Conveyor.npy")
allNormal = np.load("dev_test_ToyConveyorNormal.npy", allow_pickle=True)

#taking a sample size of 10,000
#8:2 split (training_data:testing_data ratio)
np.random.seed(1234)
np.random.shuffle(data)
training_data = data[:] #length of 8000
# testing_data = data[8000:10000] #length of 2000

windows = makeWindows(training_data, 43) #this makes the window size 11 x 10
# testWindows = makeWindows(testing_data, 5)

#creating the batches - fan
#total windows will be 247,938
BATCH_SIZE = 31
BATCHES = []

copy = windows

seed = 0

iterations = int((len(copy))/BATCH_SIZE)
for i in range(iterations):
    random.seed(seed)
    start = random.randint(0, len(copy) - BATCH_SIZE)
    seed += 1
    end = start + BATCH_SIZE
    batch = copy[start:end]
    BATCHES.append(batch)
    copy = copy[:start] + copy[end:] #taking out the batch from the training_data
    #to prevent duplicate numbers from appearing across batches
   
print(len(copy))
print(f'Shape: {len(BATCHES)}, {len(BATCHES[0])}')


#converting the python list to tensor

batched_training_data = np.array(BATCHES)
np.random.seed(2345)
np.random.shuffle(batched_training_data)
batched_training_data = torch.from_numpy(batched_training_data).type(torch.float32)

model = CAE()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

size = 31

allNormalCut = allNormal[:31, 0]
allNormalCut = makeWindows(allNormalCut, 43)
allNormalCut = allNormalCut[:size]
allNormalCut = np.array(allNormalCut)

NUM_OF_EPOCH = 50

train_losses = []
val_losses = []

#outputs = []

def test(testX):
    testX = torch.from_numpy(testX).type(torch.float32)
    outputs = model(testX.view(-1,1,11,10))
    val_loss = criterion(outputs.view(-1,1,11,10), testX.view(-1,1,11,10))
    return val_loss

with open("ToyConveyor_model1.log", "a") as f:
    for epoch in range(NUM_OF_EPOCH):
        for i in range(len(batched_training_data)):
            output = model(batched_training_data[i].view(-1,1,11,10))
            loss = criterion(output.view(-1,1,11,10), batched_training_data[i].view(-1,1,11,10))
#             train_losses.append(loss.item())
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            if i % 50 == 0:
                vloss = test(allNormalCut)
                val_losses.append(vloss)
                f.write(f"ToyConveyor_1,{round(time.time(),3)},{round(float(loss.item()), 4)},{round(float(vloss),4)}\n")
    #outputs.append((epoch, output, batched_training_data[i]))
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
#     np.save("training_Losses17.npy", train_losses)



filename = 'ToyConveyor_1.pt'
pickle.dump(model, open(filename, 'wb'))

18
Shape: 8322, 31
Epoch 0, Loss: 27.2805
Epoch 1, Loss: 24.3156
Epoch 2, Loss: 22.1905
Epoch 3, Loss: 20.1724
Epoch 4, Loss: 18.3250
Epoch 5, Loss: 16.5259
Epoch 6, Loss: 14.9356
Epoch 7, Loss: 13.3953
Epoch 8, Loss: 12.0105
Epoch 9, Loss: 10.6048
Epoch 10, Loss: 9.4082
Epoch 11, Loss: 8.3769
Epoch 12, Loss: 7.3402
Epoch 13, Loss: 6.4659
Epoch 14, Loss: 5.7478
Epoch 15, Loss: 4.9693
Epoch 16, Loss: 4.4370
Epoch 17, Loss: 3.8533
Epoch 18, Loss: 3.4017
Epoch 19, Loss: 3.0488
Epoch 20, Loss: 2.7550
Epoch 21, Loss: 2.5219
Epoch 22, Loss: 2.3365
Epoch 23, Loss: 2.1740
Epoch 24, Loss: 2.0505
Epoch 25, Loss: 1.9727
Epoch 26, Loss: 1.9072
Epoch 27, Loss: 1.9086
Epoch 28, Loss: 1.8365
Epoch 29, Loss: 1.8037
Epoch 30, Loss: 1.7898
Epoch 31, Loss: 1.8032
Epoch 32, Loss: 1.7782
Epoch 33, Loss: 1.7779
Epoch 34, Loss: 1.7735
Epoch 35, Loss: 1.7661
Epoch 36, Loss: 1.7630
Epoch 37, Loss: 1.7561
Epoch 38, Loss: 1.7557
Epoch 39, Loss: 1.7419
Epoch 40, Loss: 1.7363
Epoch 41, Loss: 1.7323
Epoch 42, Loss:

In [17]:
#training
data = np.load("dev_valve.npy")
allNormal = np.load("dev_test_valveNormal.npy", allow_pickle=True)

#taking a sample size of 10,000
#8:2 split (training_data:testing_data ratio)
np.random.seed(1234)
np.random.shuffle(data)
training_data = data[:] #length of 8000
# testing_data = data[8000:10000] #length of 2000

windows = makeWindows(training_data, 43) #this makes the window size 11 x 10
# testWindows = makeWindows(testing_data, 5)

#creating the batches - fan
#total windows will be 247,938
BATCH_SIZE = 31
BATCHES = []

copy = windows

seed = 0

iterations = int((len(copy))/BATCH_SIZE)
for i in range(iterations):
    random.seed(seed)
    start = random.randint(0, len(copy) - BATCH_SIZE)
    seed += 1
    end = start + BATCH_SIZE
    batch = copy[start:end]
    BATCHES.append(batch)
    copy = copy[:start] + copy[end:] #taking out the batch from the training_data
    #to prevent duplicate numbers from appearing across batches
   
print(len(copy))
print(f'Shape: {len(BATCHES)}, {len(BATCHES[0])}')


#converting the python list to tensor

batched_training_data = np.array(BATCHES)
np.random.seed(2345)
np.random.shuffle(batched_training_data)
batched_training_data = torch.from_numpy(batched_training_data).type(torch.float32)

model = CAE()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

size = 31

allNormalCut = allNormal[:31, 0]
allNormalCut = makeWindows(allNormalCut, 43)
allNormalCut = allNormalCut[:size]
allNormalCut = np.array(allNormalCut)

NUM_OF_EPOCH = 50

train_losses = []
val_losses = []

#outputs = []

def test(testX):
    testX = torch.from_numpy(testX).type(torch.float32)
    outputs = model(testX.view(-1,1,11,10))
    val_loss = criterion(outputs.view(-1,1,11,10), testX.view(-1,1,11,10))
    return val_loss

with open("valve_model1.log", "a") as f:
    for epoch in range(NUM_OF_EPOCH):
        for i in range(len(batched_training_data)):
            output = model(batched_training_data[i].view(-1,1,11,10))
            loss = criterion(output.view(-1,1,11,10), batched_training_data[i].view(-1,1,11,10))
#             train_losses.append(loss.item())
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            if i % 50 == 0:
                vloss = test(allNormalCut)
                val_losses.append(vloss)
                f.write(f"valve_1,{round(time.time(),3)},{round(float(loss.item()), 4)},{round(float(vloss),4)}\n")
    #outputs.append((epoch, output, batched_training_data[i]))
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
#     np.save("training_Losses17.npy", train_losses)



filename = 'valve_1.pt'
pickle.dump(model, open(filename, 'wb'))

21
Shape: 8075, 31
Epoch 0, Loss: 11.3169
Epoch 1, Loss: 8.7331
Epoch 2, Loss: 7.5061
Epoch 3, Loss: 7.0863
Epoch 4, Loss: 6.9181
Epoch 5, Loss: 6.5297
Epoch 6, Loss: 6.5321
Epoch 7, Loss: 6.4034
Epoch 8, Loss: 6.2547
Epoch 9, Loss: 6.2791
Epoch 10, Loss: 6.2331
Epoch 11, Loss: 6.1817
Epoch 12, Loss: 6.0636
Epoch 13, Loss: 6.0336
Epoch 14, Loss: 5.9892
Epoch 15, Loss: 5.9627
Epoch 16, Loss: 5.9352
Epoch 17, Loss: 5.8517
Epoch 18, Loss: 5.9021
Epoch 19, Loss: 5.8999
Epoch 20, Loss: 5.9043
Epoch 21, Loss: 5.8982
Epoch 22, Loss: 5.9227
Epoch 23, Loss: 5.9573
Epoch 24, Loss: 5.9356
Epoch 25, Loss: 5.8924
Epoch 26, Loss: 5.9395
Epoch 27, Loss: 5.9344
Epoch 28, Loss: 5.9453
Epoch 29, Loss: 5.9462
Epoch 30, Loss: 5.9355
Epoch 31, Loss: 5.9283
Epoch 32, Loss: 5.9240
Epoch 33, Loss: 5.9299
Epoch 34, Loss: 5.8908
Epoch 35, Loss: 5.8953
Epoch 36, Loss: 5.8808
Epoch 37, Loss: 5.8771
Epoch 38, Loss: 5.8704
Epoch 39, Loss: 5.8632
Epoch 40, Loss: 5.8337
Epoch 41, Loss: 5.8025
Epoch 42, Loss: 5.8091
E

In [ ]:
# model = pickle.load(open('model_20.pt','rb'))
# a = 100
# #look at a = 1, it looks very similar
# output = model(torch.from_numpy(windows[a].reshape(-1,1,11,10)).type(torch.float32))
# # output2 = model(torch.from_numpy(testWindows[2000].reshape(-1,1,11,86)).type(torch.float32))

# criterion = nn.MSELoss()
# testing = criterion(output.view(-1,1,11,10), torch.from_numpy(windows[a]).type(torch.float32).view(-1,1,11,10))
# print(testing.item())

# #reconstructed
# plt.figure(figsize=(25,10))
# librosa.display.specshow(output.view(11,10).detach().numpy(), x_axis = "time", y_axis = "mel", sr=22050)

# plt.colorbar(format="%+2.f")
# plt.show()

# #actual
# plt.figure(figsize=(25,10))
# librosa.display.specshow(windows[a].reshape(11,10), x_axis = "time", y_axis = "mel", sr=22050)

# plt.colorbar(format="%+2.f")
# plt.show()

